In [16]:
%run 0.0_Dependance.ipynb

In [17]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
import numpy as np
from joblib import Parallel, delayed
from joblib import dump,load

def load_create_dftop(df_path, n_top):
    df = load(df_path)
    df_sorted = df.sort_values(by="LogLoss_mean", ascending=False)
    df_top = df_sorted.iloc[:n_top, :]
    return df_top

# Fonction pour entraîner le modèle et calculer le log_loss
def compute_log_loss(i, df, X_train, y_train, X_test, y_test):
    dico = df.iloc[i, :-2].to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    return log_loss(y_test, probs)

df_score_random_v1_top_1000 = load_create_dftop("df_score_random_v1.joblib", 1000)

# Parallélisation du calcul du log_loss
log_losses = Parallel(n_jobs=-1)(delayed(compute_log_loss)(i, df_score_random_v1_top_1000, X_train, y_train, X_test, y_test) for i in range(1000))

df_score_random_v1_top_1000['Test_log_loss'] = log_losses
df_score_random_v1_top_1000 = df_score_random_v1_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_v1_top_1000,"df_score_random_v1_top_1000.joblib")


['df_score_random_v1_top_1000.joblib']

In [28]:
df_score_random_v1_top_1000 = df_score_random_v1_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_v1_top_1000,"df_score_random_v1_top_1000.joblib")

['df_score_random_v1_top_1000.joblib']

In [29]:
df_score_random_v1_top_1000 = load('df_score_random_v1_top_1000.joblib')
df_score_random_v1_top_1000

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std,Test_log_loss
660,400,6,0.05,0.5,1,0.2,6,1,3,multi:softprob,mlogloss,gbtree,-0.430487,0.012681,0.439749
3206,700,5,0.05,0.5,1,0.2,5,0.1,7,multi:softprob,mlogloss,gbtree,-0.430669,0.013004,0.439891
3346,600,6,0.06,0.5,1,0.2,6,0.8,3,multi:softprob,mlogloss,gbtree,-0.428670,0.012491,0.440120
3234,500,6,0.05,0.5,1,0.2,8,0.7,2,multi:softprob,mlogloss,gbtree,-0.429280,0.012886,0.440231
2096,500,5,0.06,0.5,1,0.2,5,1,2,multi:softprob,mlogloss,gbtree,-0.430991,0.012360,0.440263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,900,6,0.1,0.5,0.75,0.2,6,0.6,9,multi:softprob,mlogloss,gbtree,-0.435179,0.013186,0.457103
4112,900,6,0.09,0.5,0.75,0.2,9,0.6,0,multi:softprob,mlogloss,gbtree,-0.435623,0.015754,0.457256
788,800,4,0.09,0.5,0.75,0.2,6,0.2,2,multi:softprob,mlogloss,gbtree,-0.435318,0.016194,0.457385
23,900,6,0.1,0.5,0.75,0.2,5,0.6,5,multi:softprob,mlogloss,gbtree,-0.435217,0.013732,0.459041


In [20]:
"""df_score_random = df_score_random.sort_values(by="LogLoss_mean",ascending=False)
df_score_random_top_1000 = df_score_random.iloc[:1000,:]

list_log_loss=[]
for i in range(1000):
    df = df_score_random_top_1000.iloc[i,:-2]
    dico = df.to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    logloss = log_loss(y_test, probs)
    list_log_loss.append(logloss)
    
df_score_random_top_1000['Test_log_loss']=list_log_loss"""

'df_score_random = df_score_random.sort_values(by="LogLoss_mean",ascending=False)\ndf_score_random_top_1000 = df_score_random.iloc[:1000,:]\n\nlist_log_loss=[]\nfor i in range(1000):\n    df = df_score_random_top_1000.iloc[i,:-2]\n    dico = df.to_dict()\n    model = XGBClassifier(**dico)\n    model.fit(X_train, y_train)\n    probs = model.predict_proba(X_test)\n    logloss = log_loss(y_test, probs)\n    list_log_loss.append(logloss)\n    \ndf_score_random_top_1000[\'Test_log_loss\']=list_log_loss'